In [2]:
# load the data

library(readr)
library(dplyr)
library(stringr)

crime_2005 <- data.frame(read_csv("Crime_Data_2005.csv"))
crime_2006 <- data.frame(read_csv("Crime_Data_2006.csv"))
crime_2007 <- data.frame(read_csv("Crime_Data_2007.csv"))
crime_2008 <- data.frame(read_csv("Crime_Data_2008.csv"))
crime_2009 <- data.frame(read_csv("Crime_Data_2009.csv"))
crime_2010 <- data.frame(read_csv("Crime_Data_2010.csv"))
crime_2011 <- data.frame(read_csv("Crime_Data_2011.csv"))
crime_2012 <- data.frame(read_csv("Crime_Data_2012.csv"))
crime_2013 <- data.frame(read_csv("Crime_Data_2013.csv"))
crime_2014 <- data.frame(read_csv("Crime_Data_2014.csv"))
crime_2015 <- data.frame(read_csv("Crime_Data_2015.csv"))
crime_2016 <- data.frame(read_csv("Crime_Data_2016_39.csv"))
crime_2017 <- data.frame(read_csv("Crime_Data_2017_9.csv"))

Parsed with column specification:
cols(
  INCIDENT_NUMBER = col_character(),
  DATE_REPORTED = col_datetime(format = ""),
  DATE_OCCURED = col_datetime(format = ""),
  UOR_DESC = col_character(),
  CRIME_TYPE = col_character(),
  NIBRS_CODE = col_character(),
  UCR_HIERARCHY = col_character(),
  ATT_COMP = col_character(),
  LMPD_DIVISION = col_character(),
  LMPD_BEAT = col_character(),
  PREMISE_TYPE = col_character(),
  BLOCK_ADDRESS = col_character(),
  CITY = col_character(),
  ZIP_CODE = col_double(),
  ID = col_double()
)
Parsed with column specification:
cols(
  INCIDENT_NUMBER = col_character(),
  DATE_REPORTED = col_datetime(format = ""),
  DATE_OCCURED = col_datetime(format = ""),
  UOR_DESC = col_character(),
  CRIME_TYPE = col_character(),
  NIBRS_CODE = col_character(),
  UCR_HIERARCHY = col_character(),
  ATT_COMP = col_character(),
  LMPD_DIVISION = col_character(),
  LMPD_BEAT = col_character(),
  PREMISE_TYPE = col_character(),
  BLOCK_ADDRESS = col_character(),
  CIT

In [4]:
# join all table into one data frame
ky_join <- bind_rows(list(crime_2005,crime_2006,crime_2007,crime_2008,
                          crime_2009,crime_2010,crime_2011,crime_2012,
                          crime_2013,crime_2014,crime_2015,crime_2016,crime_2017))

In [5]:
# write a function to find drug offense 

flag_drug <- function(v) {
    
    n <- length(v)
    drug <- rep(FALSE, n)
    
    for(i in 1:n){
      drug [i] <- v[i] == c("DRUGS/ALCOHOL VIOLATIONS")  
    }
       
    return(drug)

}

In [6]:
# add is_drug column to identify the drug/acohol offense
ky_join$is_drug <- flag_drug(ky_join$CRIME_TYPE)


In [7]:
# only select drug offense in drug_ky

drug_ky <- filter(ky_join,str_detect(ky_join$UOR_DESC, "ALCOHOL")==FALSE & 
                  is_drug==TRUE,UOR_DESC != "ALCOHOL")
nrow(drug_ky)

[1] 178850

In [8]:
# write a function to find not felony/ minor offense 

flag_minor <- function(v) {
    
    n <- length(v)
    minor <- rep(FALSE, n)
    
    for(i in 1:n){
        if(str_detect(v[i],"PARAPHERNALIA")){
            minor[i] <- TRUE
        }else if(str_detect(v[i],"MARIJUANA")){
            if(str_detect(v[i],"LESS THAN 8 OZ")||str_detect(v[i],"< 5 PLANTS")){
                minor[i] <- TRUE
            }
            
        }        
    }
       
    return(minor)

}


In [9]:
# add is_minor column to identify the minor drug offense
drug_ky$is_minor <- flag_minor(drug_ky$UOR_DESC)
table(drug_ky$is_minor)


 FALSE   TRUE 
118687  60163 

In [77]:
# # add year column to identify the year that violation occurred 
library(lubridate)
drug_ky$year <- year(drug_ky$DATE_OCCURED)
ky_join$year <- year(ky_join$DATE_OCCURED) 


In [78]:
# summarise offense based on year

ky_drug_minor <- drug_ky %>%
group_by(year) %>%
summarise(minor_offense=sum(is_minor),felony = sum(is_minor==FALSE),
          total=sum(is_minor)+sum(is_minor==FALSE))  

ky_all <- ky_join %>%
group_by(year) %>%
summarise(total_offense=n())  


In [79]:
# only select year 2005-2017
ky_drug_minor_1 <- ky_drug_minor[-c(1,2,3,4,5),]
ky_drug_minor_1

ky_all_1 <- ky_all[-c(1:48),]
ky_all_1

year,minor_offense,felony,total
<dbl>,<int>,<int>,<int>
2005,3701,7261,10962
2006,4225,8996,13221
2007,4902,10264,15166
2008,3927,8137,12064
2009,4091,8451,12542
2010,4569,8839,13408
2011,4813,8583,13396
2012,5233,10841,16074
2013,5360,9568,14928


year,total_offense
<dbl>,<int>
2005,77351
2006,83865
2007,90795
2008,85858
2009,77991
2010,82990
2011,85935
2012,81175
2013,76891


In [23]:
# load rdd package
install.packages("rdd")
library(rdd)
library(ggplot2)

also installing the dependencies ‘RcppEigen’, ‘pbkrtest’, ‘lme4’, ‘car’, ‘AER’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: sandwich
Loading required package: lmtest
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: AER
Loading required package: car
Loading required package: carData

Attaching package: ‘car’

The following object is masked from ‘package:dplyr’:

    recode

Loading required package: survival
Loading required package: Formula


In [24]:
# RD estimate for minor drug offense 
reg.1 = RDestimate(minor_offense~year,data=ky_drug_minor_1,cutpoint = 2012)

In [73]:
summary(reg.1)

jpeg('minorplot.jpg')
plot(reg.1)
dev.off()


Call:
RDestimate(formula = minor_offense ~ year, data = ky_drug_minor_1, 
    cutpoint = 2012)

Type:
sharp 

Estimates:
           Bandwidth  Observations  Estimate  Std. Error  z value  Pr(>|z|)    
LATE        6.609     12            581.3     266.5       2.181    0.029152  * 
Half-BW     3.304      7            142.7     109.6       1.302    0.193070    
Double-BW  13.217     13            613.6     222.2       2.762    0.005742  **
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

F-statistics:
           F       Num. DoF  Denom. DoF  p      
LATE        7.111  3         8           0.02407
Half-BW    15.044  3         3           0.05183
Double-BW   7.417  3         9           0.01669


png 
  2

In [62]:
# RD estimate for felony drug offense 
reg.2 = RDestimate(felony~year,data=ky_drug_minor_1,cutpoint = 2012)

In [76]:
#summary(reg.2)

jpeg('felonyplot.jpg')
plot(reg.2,xlab="Year",ylab="Cases")
dev.off()

png 
  2

In [72]:
jpeg('totalplot.jpg')

barplot(ky_all_1$total_offense,main="Total crime number in Louisville over Year", 
        xlab = "Year",ylab="Number of Case", names.arg =c("2005","2006","2007","2008","2009","2010",
                                    "2011","2012","2013","2014","2015","2016","2017","NA"))

dev.off()

png 
  2